<a href="https://colab.research.google.com/github/kutaydemiralay/MPC-Style-SCP-for-Quadrotor-Obstacle-Avoidance-and-Controller-Design/blob/main/KutayDemiralay_Quadrotor_Obstacle_Avoidance_SimulinkData_Censored.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Sequential Convex Programming for Quadrotor Obstacle Avoidance**

In [ ]:
#For now since this is not published yet the code is censored, you can reach kutay@uw.edu for the code.

**Exporting the Trajectory and Control Inputs from SCP Algorithm to Simulink Simulation**

In [ ]:
import numpy as np
params = parameters()
wind_vector = np.array([0.0, 0.0, 0.0])  # No wind
result = main(params, wind_vector=wind_vector)


# Assuming result['drone_state'] is a 305x13 array
drone_state = result['drone_state']

# Check the shape of drone_state
print("drone_state shape:", drone_state.shape)  # Should print (305, 13)

# Transpose drone_state to get the correct shape (13, 305)
drone_state = drone_state.T  # Now it should be 13x305

# Check the shape after transposing
print("Transposed drone_state shape:", drone_state.shape)  # Should print (13, 305)

# Extract the quaternion part (columns 6 to 9 are q0, q1, q2, q3)
quaternions = drone_state[6:10, :]  # Now quaternions should be 4x305
print("quaternions shape:", quaternions.shape)  # Should print (4, 305)

# Initialize yawpitchroll to have 3 rows (yaw, pitch, roll) and 305 columns (time steps)
yawpitchroll = np.zeros((3, quaternions.shape[1]))

# Function to convert quaternion to Euler angles (yaw, pitch, roll)
def quat_to_euler(q):
    q0, q1, q2, q3 = q

    yaw = np.arctan2(2 * (q0 * q3 + q1 * q2), 1 - 2 * (q2**2 + q3**2))
    pitch = np.arcsin(2 * (q0 * q2 - q3 * q1))
    roll = np.arctan2(2 * (q0 * q1 + q2 * q3), 1 - 2 * (q1**2 + q2**2))

    return pitch, yaw, roll

# Convert quaternions to yaw, pitch, and roll for each time step
for i in range(quaternions.shape[1]):
    pitch, yaw, roll = quat_to_euler(quaternions[:, i])
    yawpitchroll[:, i] = [yaw, pitch, roll]  # Store yaw, pitch, roll in this order

# Check the final shape of yawpitchroll
print("yawpitchroll shape:", yawpitchroll.shape)  # Should print (3, 305)

# Delete simulinkdata if it exists
if 'simulinkdata' in locals():
    del simulinkdata

# Assume params is already defined and contains the initial and final states
params = parameters()  # Call your parameters() function to get the params dictionary

# Get the initial and final states from params
initial_state = params['initial_state']  # Pull initial state from params
final_state = params['final_state']      # Pull final state from params
hovering_altitude = 2  # Predefined hovering altitude (positive)

# Number of points (305 points correspond to 8 seconds)
num_points_8s = drone_state.shape[1]  # This should be 305 based on your data
num_points_5s = int((num_points_8s / 8) * 5)  # Dynamically calculate points for the first 5 seconds
num_points_hover = int((num_points_8s / 8) * 5)  # Points for 5 seconds of hovering

# Create the time vector for the full 13 seconds (first 5 seconds + original 8 seconds)
time_5s = np.linspace(0, 5, num_points_5s)  # Points for the first 5 seconds
time_8s = np.linspace(5, 13, num_points_8s)  # Points for the remaining 8 seconds
time_hover = np.linspace(13, 18, num_points_hover)  # Points for the 5 seconds of hovering at the end

# Extract x, y, z, yaw, pitch, roll data from the original data
x_8s = drone_state[0, :] - initial_state[0]  # Shift x_8s to account for the initial x value
y_8s = drone_state[1, :]
z_8s = drone_state[2, :]
yaw_8s = yawpitchroll[0, :]
pitch_8s = yawpitchroll[1, :]
roll_8s = yawpitchroll[2, :]

# Negate the Z values as requested (subtract hovering altitude)
z_8s = -z_8s

# Create new x, y, z, yaw, pitch, roll for the first 5 seconds (all zeros)
x_5s = np.zeros(num_points_5s)
y_5s = np.zeros(num_points_5s)
z_5s = np.zeros(num_points_5s)  # Z values are zeros for the first 5 seconds
yaw_5s = np.zeros(num_points_5s)
pitch_5s = np.zeros(num_points_5s)
roll_5s = np.zeros(num_points_5s)

# Calculate the hover state (x_hover = -initial_state[0] + final_state[0], z_hover = -hovering_altitude)
x_hover = (-initial_state[0] + final_state[0]) * np.ones(num_points_hover)
y_hover = np.zeros(num_points_hover)
z_hover = np.zeros(num_points_hover)  # Hover at the negative hovering altitude
yaw_hover = np.zeros(num_points_hover)
pitch_hover = np.zeros(num_points_hover)
roll_hover = np.zeros(num_points_hover)

# Combine the data for the first 5 seconds, the original 8 seconds, and the final 5 seconds of hovering
time = np.concatenate([time_5s, time_8s, time_hover])  # Time now goes from 0 to 18 seconds
x = np.concatenate([x_5s, x_8s, x_hover])
y = np.concatenate([y_5s, y_8s, y_hover])
z = np.concatenate([z_5s, z_8s, z_hover]) - hovering_altitude  # Adjust all Z values by subtracting the hovering altitude
yaw = np.concatenate([yaw_5s, yaw_8s, yaw_hover])
pitch = np.concatenate([pitch_5s, pitch_8s, pitch_hover])
roll = np.concatenate([roll_5s, roll_8s, roll_hover])

# Create the simulinkdata array with 7 columns (Time, X, Y, Z, Yaw, Pitch, Roll)
simulinkdata = np.vstack([time, x, y, z, yaw, pitch, roll]).T

# Check the final shape of simulinkdata
print("simulinkdata shape:", simulinkdata.shape)  # Should print (495 + num_points_hover, 7)




Iteration Number
-----------------


/usr/local/lib/python3.10/dist-packages/cvxpy/reductions/solvers/solving_chain.py:356: FutureWarning:


    You specified your problem should be solved by ECOS. Starting in
    CXVPY 1.6.0, ECOS will no longer be installed by default with CVXPY.
    Please either add ECOS as an explicit install dependency to your project
    or switch to our new default solver, Clarabel, by either not specifying a
    solver argument or specifying ``solver=cp.CLARABEL``. To suppress this
    warning while continuing to use ECOS, you can filter this warning using
    Python's ``warnings`` module until you are using 1.6.0.
    



   0
   1
   2
   3
   4
   5
   6
   7
   8
   9
  10
  11
  12
  13
  14
  15
  16
  17
  18
  19
  20
Dimension of x_full: (305, 13)
drone_state shape: (305, 13)
Transposed drone_state shape: (13, 305)
quaternions shape: (4, 305)
yawpitchroll shape: (3, 305)
simulinkdata shape: (685, 7)


In [ ]:
import pandas as pd
from google.colab import files
!pip install xlsxwriter

# Create a DataFrame from simulinkdata
simulinkdata_df = pd.DataFrame(simulinkdata, columns=['Time', 'X', 'Y', 'Z', 'Yaw', 'Pitch', 'Roll'])

# Save the DataFrame as an Excel (.xlsx) file with the sheet name 'data'
with pd.ExcelWriter('simulinkdata.xlsx', engine='xlsxwriter') as writer:
    simulinkdata_df.to_excel(writer, sheet_name='data', index=False)

# Download the file in Colab
files.download('simulinkdata.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Wind Added to algorithm to Sequential Convex Programming for Quadrotor Obstacle Avoidance**

In [ ]:
def run_quadrotor_simulation_with_wind(wind=True):  # Change wind condition here
    # Initialize the parameters
    params = parameters()

    # Define the wind vector based on the wind parameter
    if wind:
        wind_vector = np.array([0.0, 0.10, 0.0])  # Constant Wind is present
    else:
        wind_vector = np.array([0.0, 0.0, 0.0])  # No wind

    # Run the main PTR function to get the result with or without wind
    result = main(params, wind_vector=wind_vector)

    # Plot the 3D static plot with node points and print the number of nodes
    plot_static_3d_with_nodes_and_count(result, wind, params, wind_vector)

    # Print the total fuel cost and node violation
    nonlinear_min_fuel_cost(result, "Quadrotor Simulation with Wind" if wind else "Quadrotor Simulation without Wind",
                            params)


# Call the function to run the simulation with or without wind
run_quadrotor_simulation_with_wind()

Iteration Number
-----------------


/usr/local/lib/python3.10/dist-packages/cvxpy/reductions/solvers/solving_chain.py:356: FutureWarning:


    You specified your problem should be solved by ECOS. Starting in
    CXVPY 1.6.0, ECOS will no longer be installed by default with CVXPY.
    Please either add ECOS as an explicit install dependency to your project
    or switch to our new default solver, Clarabel, by either not specifying a
    solver argument or specifying ``solver=cp.CLARABEL``. To suppress this
    warning while continuing to use ECOS, you can filter this warning using
    Python's ``warnings`` module until you are using 1.6.0.
    



   0
   1
   2
   3
   4
   5
   6
   7
   8
   9
  10
  11
  12
  13
  14
  15
  16
  17
  18
  19
  20
Dimension of x_full: (305, 13)
Total number of nodes: 20


Total fuel cost for Quadrotor Simulation with Wind is 22.133451420968235
Total Node Violation: 41.71118972751173


**MPC style SCP algorithm**

In [ ]:
def run_quadrotor_simulation_with_wind():
    # Initialize the parameters
    params = parameters()

    # Define the wind vector for the simulation
    wind_vector = np.array([0.0, 0.0, 0.0])  # Wind is present

    # Run the MPC-style SCP over the trajectory
    for node in range(int(params['total_time'] / params['dt_state_space'])):
        print(f"Running SCP for Node {node}")

        # Reset final state to the desired state
        params['final_state'] = np.array([10, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])

        # Generate initial straight-line trajectory and control guesses for the current node
        x_bar, u_bar = straight_line_init(params['initial_state'], params['initial_control'], params['final_state'], params['n'], params)

        # Run the main function to get the result for the current node
        result = main(params, wind_vector=wind_vector)

        # Print the total time for the current iteration
        print(f"Total time remaining for Node {node}: {params['total_time']} seconds")

        # Plot the 3D static plot with node points and print the number of nodes
        plot_static_3d_with_nodes_and_count(result, True, params, wind_vector)

        # Print the total fuel cost and node violation
        nonlinear_min_fuel_cost(result, f"Quadrotor Simulation at Node {node} with Wind", params)

        # Update initial state for the next node using the second node's state from the current result
        initial_state = result['drone_state'][(int(params['dt_state_space'] / params['dt_simulation']))]

        # Update parameters for the next iteration
        params['initial_state'] = initial_state
        params['total_time'] -= params['dt_state_space']  # Reduce total time by the discretization step

# Call the function to run the simulation with wind
run_quadrotor_simulation_with_wind()